In [1]:
from typing import TYPE_CHECKING


if TYPE_CHECKING:
    from math_rag.application.containers import ApplicationContainer
    from math_rag.infrastructure.containers import InfrastructureContainer

    application_container: ApplicationContainer
    infrastructure_container: InfrastructureContainer

In [2]:
RESET = False
%load_ext hooks.notebook_hook

2025-07-15 23:52:49,117 - INFO - datasets - config.py:54 - PyTorch version 2.6.0 available.


### Playground

In [ ]:
from agents import Agent, Runner


math_problem_solver_agent = Agent(
    name='Assistant',
    instructions='You are a helpful assistant',
    output_type=str,
)

result = await Runner.run(
    math_problem_solver_agent, 'Write a haiku about recursion in programming.'
)
print(result.final_output)

Code repeats itself,  
Functions within functions loop—  
Infinite echoes.


In [ ]:
TODO = ...

math_problem_solver_agent = Agent(
    name=TODO,
    instructions=TODO,
    prompt=TODO,
    handoff_description=TODO,
    handoffs=TODO,
    model=TODO,
    model_settings=TODO,
    tools=TODO,
    mcp_servers=TODO,
    mcp_config=TODO,
    input_guardrails=TODO,
    output_guardrails=TODO,
    output_type=TODO,
    hooks=TODO,
    tool_use_behavior=TODO,
    reset_tool_choice=TODO,
)

In [ ]:
# TODO use openai guardrail model

In [ ]:
from agents import Handoff
from agents.extensions.handoff_filters import remove_all_tools


tool: FunctionTool = ...
math_problem_solver_agent: Agent = ...

handoff = Handoff(
    tool_name=tool.name,
    tool_description=tool.description,
    input_json_schema=tool.params_json_schema,
    on_invoke_handoff=tool.on_invoke_tool,
    agent_name=math_problem_solver_agent.name,
    input_filter=remove_all_tools,
    strict_json_schema=True,
    is_enabled=True,
)

In [ ]:
from agents import HandoffInputData


def drop_pre_handoff_items(data: HandoffInputData) -> HandoffInputData:
    """
    Remove everything generated before the handoff turn,
    so the next agent only sees the original history + new_items.
    """
    return HandoffInputData(
        input_history=data.input_history,
        pre_handoff_items=(),
        new_items=data.new_items,
    )


def keep_last_5_new_items(data: HandoffInputData) -> HandoffInputData:
    """
    Keep only the last 5 items from new_items,
    preserving full history and pre_handoff_items.
    """
    return HandoffInputData(
        input_history=data.input_history,
        pre_handoff_items=data.pre_handoff_items,
        new_items=tuple(data.new_items[-5:]),
    )

### Jupyter tool

In [3]:
jupyter_client = infrastructure_container.jupyter_client()

In [4]:
await jupyter_client.start_session(user_id='0')

JupyterStartSessionResult(message='Session started successfully', notebook_path='/mnt/jupyter_sessions/0/notebook_0.ipynb')

In [ ]:
from agents import Agent, Runner


math_problem_solver_agent = Agent(
    name='CodeRunner',
    instructions='Use the execute_code tool to run Python code snippets.',
    tools=[execute_code_tool],
)

result = await Runner.run(
    math_problem_solver_agent, 'Please run some Python code, invent it on your own'
)
result.final_output

'Here are 10 random integers between 1 and 100:\n\\[69, 74, 94, 5, 33, 70, 18, 40, 26, 50\\]'

In [9]:
for x in result.new_items:
    if hasattr(x, 'output'):
        print(x.output)

{"output":"[69, 74, 94, 5, 33, 70, 18, 40, 26, 50]","error":null}


In [ ]:
await jupyter_client.end_session(user_id='0')

### Katex tool

In [ ]:
from agents import Agent, Runner


math_problem_solver_agent = Agent(
    name='CodeRunner',
    instructions='Use the validate_katex tool to validate KaTeX.',
    tools=[validate_katex_tool],
)

result = await Runner.run(
    math_problem_solver_agent, 'Please validate some KaTeX, invent it on your own'
)
result.final_output

'The KaTeX expression \\( c = \\pm\\sqrt{a^2 + b^2} \\) is valid.'

### Graph tool

In [ ]:
from agents import Agent, Runner


math_problem_solver_agent = Agent(
    name='CodeRunner',
    instructions='Use the search_graph tool to search the knowledge graph.',
    tools=[search_graph_tool],
)

result = await Runner.run(
    math_problem_solver_agent, 'Please search graph, send a query about logistic regression'
)
result.final_output

"Here are some details about logistic regression derived from the search graph:\n\n1. **Function Minimization**:\n   - The function \\( f(\\mathbf{x}) \\) is minimized using methods like Newton's method, where the function is evaluated at the current point in the parameter space.\n   - The quadratic approximation at a point is derived from the Taylor series expansion around that point.\n\n2. **Taylor Series Expansion**:\n   - For functions of a single variable, like \\( f(x) \\), the function's quadratic approximation is derived using its Taylor series.\n   - The point \\( a \\) serves as the center of the expansion, from which the function is approximated.\n\n3. **Initial Point in Optimization**:\n   - The symbol \\( x_{0} \\) denotes the initial point for starting the minimization process using Newton's method.\n\n4. **Approximation of Functions**:\n   - The Taylor series provides a way to approximate functions at a point.\n   - Specifically, the second-order Taylor expansion offers 

### Web search tool

In [5]:
from agents import Agent, Runner, WebSearchTool


math_problem_solver_agent = Agent(
    name='CodeRunner',
    instructions='Use the web search tool.',
    tools=[WebSearchTool(search_context_size='low')],
)

result = await Runner.run(math_problem_solver_agent, 'search for the Pythagorean theorem')
result.final_output

'The Pythagorean theorem is a fundamental principle in geometry that describes the relationship between the sides of a right-angled triangle. It states that the square of the length of the hypotenuse (the side opposite the right angle) is equal to the sum of the squares of the lengths of the other two sides. This relationship is expressed algebraically as:\n\n\\[ a^2 + b^2 = c^2 \\]\n\nwhere \\( a \\) and \\( b \\) are the lengths of the legs (the sides forming the right angle), and \\( c \\) is the length of the hypotenuse.\n\n**Example:**\n\nConsider a right-angled triangle with legs measuring 3 units and 4 units. To find the length of the hypotenuse:\n\n\\[ 3^2 + 4^2 = c^2 \\]\n\n\\[ 9 + 16 = c^2 \\]\n\n\\[ 25 = c^2 \\]\n\nTaking the square root of both sides:\n\n\\[ c = \\sqrt{25} = 5 \\]\n\nTherefore, the hypotenuse measures 5 units.\n\n**Historical Context:**\n\nAlthough named after the ancient Greek mathematician Pythagoras (c. 570–500/490 BCE), evidence suggests that the relati

## Example solutions

In [10]:
import json

from pathlib import Path


solutions_dir_path = Path.cwd() / '..' / '.tmp' / 'solutions'
solutions_dir_path.mkdir(parents=True, exist_ok=True)

solutions = []

for json_file in solutions_dir_path.glob('*.json'):
    with open(json_file, 'r', encoding='utf-8') as f:
        solutions.append(json.load(f))

In [11]:
display(Latex(solutions[0]['math_problem_solution']['text']))

<IPython.core.display.Latex object>

In [1]:
from io import BytesIO

import requests

from PIL import Image


# Your long, encoded URL
image_url = 'http://localhost:9001/api/v1/download-shared-object/aHR0cDovLzEyNy4wLjAuMTo5MDAwL3RtcG9iamVjdC9TY3JlZW5zaG90JTIwMjAyNS0wNy0xNiUyMGF0JTIwMTAuMTQuMTYucG5nP1gtQW16LUFsZ29yaXRobT1BV1M0LUhNQUMtU0hBMjU2JlgtQW16LUNyZWRlbnRpYWw9V05JQzMzRUlRODdBVktMWDZPVFElMkYyMDI1MDcxNiUyRnVzLWVhc3QtMSUyRnMzJTJGYXdzNF9yZXF1ZXN0JlgtQW16LURhdGU9MjAyNTA3MTZUMDgyNzUwWiZYLUFtei1FeHBpcmVzPTQzMTk5JlgtQW16LVNlY3VyaXR5LVRva2VuPWV5SmhiR2NpT2lKSVV6VXhNaUlzSW5SNWNDSTZJa3BYVkNKOS5leUpoWTJObGMzTkxaWGtpT2lKWFRrbERNek5GU1ZFNE4wRldTMHhZTms5VVVTSXNJbVY0Y0NJNk1UYzFNalk1TmpnMk5Td2ljR0Z5Wlc1MElqb2lZV1J0YVc0aWZRLldmZklKWXJ0VWJMTUJBeEhVX21rSkFWUy1oOXoyZ1JDQVJqb1VXWHl6VlM2cG1KMEVQcDNfaVpUdFZHSDU1VWtfVkRxRkdHbS01SmZldmRBMkdfSEt3JlgtQW16LVNpZ25lZEhlYWRlcnM9aG9zdCZ2ZXJzaW9uSWQ9bnVsbCZYLUFtei1TaWduYXR1cmU9YTExMzZkYjEyMjY1NDUyYzJhOGFiNmQ1NTI5YWI4MTdiMjY5NGEzNzAzZDRlNWI5NWRjZjQyMDQ5ODhjOTMxMQ'

# Fetch the image from the URL
response = requests.get(image_url)
response.raise_for_status()  # Ensure we got a valid response

# Load into memory as an image
image = Image.open(BytesIO(response.content))

# Optionally show or save the image
image.show()  # Or image.save('image.png')

KeyboardInterrupt: 